In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
train_data,validation_data,test_data=tfds.load(name="imdb_reviews",
                                               split=("train[:60%]","train[60%:]",'test'),
                                               as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\yalam\tensorflow_datasets\imdb_reviews\plain_text\incomplete.QKOST8_1.0.0\imdb_reviews-trai…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\yalam\tensorflow_datasets\imdb_reviews\plain_text\incomplete.QKOST8_1.0.0\imdb_reviews-test…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\yalam\tensorflow_datasets\imdb_reviews\plain_text\incomplete.QKOST8_1.0.0\imdb_reviews-unsu…

Dataset imdb_reviews downloaded and prepared to C:\Users\yalam\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
train_example_batch,train_labels_batch=next(iter(train_data.batch(10)))

In [5]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [7]:
embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer=hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)

In [8]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [13]:
# Load the TensorFlow Hub model using the specified URL
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1", trainable=False)

# Initialize a Sequential model
model = tf.keras.Sequential()

# Wrap the hub layer with a Lambda layer and add it to the model
model.add(tf.keras.layers.Lambda(lambda x: hub_layer(x)))

# Add additional layers
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [14]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [16]:
history=model.fit(train_data.shuffle(10000).batch(100),epochs=25,
                  validation_data=validation_data.batch(100),verbose=1)

Epoch 1/25



150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5066 - loss: 1.1272 - val_accuracy: 0.5873 - val_loss: 0.6857
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5984 - loss: 0.6732 - val_accuracy: 0.6245 - val_loss: 0.6373
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6283 - loss: 0.6384 - val_accuracy: 0.6492 - val_loss: 0.6136
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6471 - loss: 0.6114 - val_accuracy: 0.6652 - val_loss: 0.5985
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6627 - loss: 0.5983 - val_accuracy: 0.6581 - val_loss: 0.5903
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6704 - loss: 0.5850 - val_accuracy: 0.6784 - val_loss: 0.5798
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6767 - loss: 0.5782 - val_accuracy: 0.6714 - val_loss: 0.5743
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6888 - loss: 0.5666 - val_accuracy: 0.6631 - v

In [ ]:
results=model.evaluate(test_data.batch(100),verbose=2)
for name,value in zip(model.metrics_names,results):
  print("%s: %.3f"%(name,value))

250/250 - 3s - loss: 0.6540 - accuracy: 0.8412 - 3s/epoch - 13ms/step
loss: 0.654
accuracy: 0.841


In [18]:
import tkinter as tk
from tkinter import ttk

def start_training():
    # Add your model training logic here
    print("Training started...")

root = tk.Tk()
root.title("IMDB Sentiment Analysis - Model Trainer")

# Model Configuration
config_frame = tk.LabelFrame(root, text="Model Configuration")
config_frame.pack(fill="both", expand="yes", padx=10, pady=10)

tk.Label(config_frame, text="Embedding Model URL:").grid(row=0, column=0)
embedding_url = tk.Entry(config_frame, width=50)
embedding_url.insert(0, "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
embedding_url.grid(row=0, column=1)

trainable_var = tk.IntVar()
tk.Checkbutton(config_frame, text="Make Embedding Trainable", variable=trainable_var).grid(row=1, columnspan=2)

tk.Label(config_frame, text="First Dense Layer Units:").grid(row=2, column=0)
dense1_units = tk.Spinbox(config_frame, from_=1, to=128, width=5)
dense1_units.grid(row=2, column=1)

tk.Label(config_frame, text="Second Dense Layer Units:").grid(row=3, column=0)
dense2_units = tk.Spinbox(config_frame, from_=1, to=128, width=5)
dense2_units.grid(row=3, column=1)

# Training & Evaluation
train_frame = tk.LabelFrame(root, text="Training & Evaluation")
train_frame.pack(fill="both", expand="yes", padx=10, pady=10)

tk.Label(train_frame, text="Number of Epochs:").grid(row=0, column=0)
epochs = tk.Spinbox(train_frame, from_=1, to=100, width=5)
epochs.grid(row=0, column=1)

shuffle_var = tk.IntVar(value=1)
tk.Checkbutton(train_frame, text="Shuffle Training Data", variable=shuffle_var).grid(row=1, columnspan=2)

tk.Label(train_frame, text="Batch Size:").grid(row=2, column=0)
batch_size = tk.Spinbox(train_frame, from_=1, to=500, width=5)
batch_size.grid(row=2, column=1)

train_button = tk.Button(train_frame, text="Start Training", command=start_training)
train_button.grid(row=3, columnspan=2)

progress = ttk.Progressbar(train_frame, orient="horizontal", length=200, mode="determinate")
progress.grid(row=4, columnspan=2)

root.mainloop()


Training started...
Training started...
Training started...
Training started...
Training started...
Training started...
